In [ ]:
!pip install tensorflow
!pip install gpflow==2.9.1

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 380.6/380.6 kB 15.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.8/45.8 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 111.0/111.0 kB 7.7 MB/s eta 0:00:00


In [ ]:
import numpy as np
import gpflow
from scipy.stats import norm
import matplotlib.pyplot as plt
from matplotlib.ticker import FuncFormatter, MaxNLocator
import pandas as pd
import itertools

In [ ]:
from google.colab import drive
import os

In [ ]:
# Mount Google Drive
drive.mount('/content/drive')

# Define the directory in Google Drive where you want to save the results
save_dir = '/content/drive/MyDrive/Vanilla_GP_Results_2D'
os.makedirs(save_dir, exist_ok=True)

To add noise to data points

In [ ]:
# Function to generate noisy data based on factorial design
def generate_noisy_data(X, n_points=5, noise_std_range=(0.0, 1.0)):
    X_noisy = []
    y_noisy = []
    noise_levels = []

    for point in X:
        x1, x2 = point  # Unpack the point into x1 and x2
        noise_level = np.random.uniform(noise_std_range[0], noise_std_range[1])
        for _ in range(n_points):
            noisy_y = calculate_y(x1, x2) + np.random.normal(0, noise_level)
            X_noisy.append([x1, x2])
            y_noisy.append(noisy_y)
            noise_levels.append(noise_level)

    X_noisy = np.array(X_noisy)
    y_noisy = np.array(y_noisy).reshape(-1, 1)
    noise_levels = np.array(noise_levels)

    return X_noisy, y_noisy, noise_levels


In [ ]:
# Function to generate a grid of points for prediction
def generate_grid(x1_limits, x2_limits, num_points=100):
    x1 = np.linspace(x1_limits[0], x1_limits[1], num_points)
    x2 = np.linspace(x2_limits[0], x2_limits[1], num_points)
    X1, X2 = np.meshgrid(x1, x2)
    grid = np.vstack([X1.ravel(), X2.ravel()]).T
    return grid, X1, X2

# Function to calculate y based on the Three Hump Camel Function
def calculate_y(X1, X2):
    X1, X2 = X[:, 0], X[:, 1]  # Extract X1 and X2 from the input array
    term1 = 2 * X1 ** 2
    term2 = -1.05 * X1 ** 4
    term3 = X1 ** 6 / 6
    term4 = X1 * X2
    term5 = X2 ** 2
    return term1 + term2 + term3 + term4 + term5

# DEFINE GP MODELS + TRAINING
def train_gp_model(X, y):
    kernel = gpflow.kernels.SquaredExponential(lengthscales=[0.1, 0.1]) # Adjust for two dimensions
    model = gpflow.models.GPR(data=(X, y), kernel=kernel, mean_function=None)
    model.likelihood.variance.assign(1e-5)  # Assign a very small value to the variance, e.g., 1e-5 | 0.001
    gpflow.utilities.set_trainable(model.likelihood.variance, False)
    opt = gpflow.optimizers.Scipy()
    opt.minimize(model.training_loss, model.trainable_variables)
    return model

# Expected Improvement function for minimization
def expected_improvement(y_best, f_mean, f_var):
    variance = np.maximum(f_var, 1e-9)
    std_dev = np.sqrt(variance)
    delta = y_best - f_mean
    with np.errstate(divide='ignore'):
        Z = delta / std_dev
        ei = delta * norm.cdf(Z) + std_dev * norm.pdf(Z)
        ei[std_dev == 0.0] = 0.0
    return ei

# MAIN SCRIPT
x1_limits = [-2.0, 2.0]  # Limits for x1
x2_limits = [-2.0, 2.0]  # Limits for x2

# Define the levels for x1 and x2
x1_manual = np.array([-2.0, -0.5, 1.0])
x2_manual = np.array([-2.0, -0.5, 1.0])

# Generate the Cartesian product of x1_manual and x2_manual using itertools
X_manual = np.array(list(itertools.product(x1_manual, x2_manual)))

# Generate the initial data
X = X_manual
y = calculate_y(X)

# Train GP model
model = train_gp_model(X, y)

# Create a DataFrame to store results
results_df = pd.DataFrame(columns=['iteration', 'x1', 'x2', 'y', 'ei', 'LCB', 'y_best', 'true_var', 'pred_var'])

# Run iterations
num_iterations = 20
for iteration in range(num_iterations):
    # Generate grid for prediction
    X_plot, X1, X2 = generate_grid(x1_limits, x2_limits, 100)  # 50 points per dimension

    # Predict f_mean and f_var for the model
    f_mean, f_var = model.predict_f(X_plot)
    f_mean = f_mean.numpy().reshape(X1.shape)
    f_var = f_var.numpy().reshape(X1.shape)

    # Calculate EI and LCB
    y_best = np.min(y)
    ei = expected_improvement(y_best, f_mean, f_var).reshape(X1.shape)
    LCB = (f_mean - 1.96 * np.sqrt(f_var)).reshape(X1.shape)
    UCB = (f_mean + 1.96 * np.sqrt(f_var)).reshape(X1.shape)

    # Calculate true variance
    y_true = calculate_y(X_plot).reshape(X1.shape)
    true_var = np.var(y_true - f_mean)

    # Find the point that maximizes EI
    max_ei_index = np.unravel_index(np.argmax(ei), ei.shape)
    x1_new_ei, x2_new_ei = X1[max_ei_index], X2[max_ei_index]
    y_new_ei = calculate_y(np.array([[x1_new_ei, x2_new_ei]]))[0]

    # Find the point with the lowest LCB
    min_lcb_index = np.unravel_index(np.argmin(LCB), LCB.shape)
    x1_new_lcb, x2_new_lcb = X1[min_lcb_index], X2[min_lcb_index]
    y_new_lcb = calculate_y(np.array([[x1_new_lcb, x2_new_lcb]]))[0]
    lcb_value = LCB[min_lcb_index]
    pred_var = f_var[min_lcb_index]

    # Update the training data with the new points
    X = np.vstack((X, [[x1_new_ei, x2_new_ei]]))
    y = np.vstack((y, y_new_ei))
    X = np.vstack((X, [[x1_new_lcb, x2_new_lcb]]))
    y = np.vstack((y, y_new_lcb))

    # Re-train the GP model
    model = train_gp_model(X, y)

    # Create a DataFrame for the new rows and concatenate it to results_df
    new_rows = pd.DataFrame([
        {
            'iteration': iteration,
            'x1': x1_new_ei,
            'x2': x2_new_ei,
            'y': y_new_ei,
            'ei': ei[max_ei_index],
            'LCB': None,  # EI point does not have an LCB value
            'y_best': round(y_best, 2),
            'true_var': true_var,
            'pred_var': None,  # EI point does not have a predicted variance value
        },
        {
            'iteration': iteration,
            'x1': x1_new_lcb,
            'x2': x2_new_lcb,
            'y': y_new_lcb,
            'ei': ei[min_lcb_index],
            'LCB': lcb_value,  # Save the LCB value
            'y_best': round(y_best, 2),
            'true_var': true_var,
            'pred_var': pred_var,  # Save the predicted variance value
        }
    ])
    results_df = pd.concat([results_df, new_rows], ignore_index=True)

    # Print the new points
    print(f"New EI point added: x1 = {x1_new_ei}, x2 = {x2_new_ei}, y = {y_new_ei}, ei = {ei[max_ei_index]}, y_best = {round(y_best, 2)}")
    print(f"New LCB point added: x1 = {x1_new_lcb}, x2 = {x2_new_lcb}, y = {y_new_lcb}, ei = {ei[min_lcb_index]}, LCB = {lcb_value}, y_best = {round(y_best, 2)}")

    # Save contour plots for EI, LCB, and Mean after each iteration
    fig, ax = plt.subplots(1, 3, figsize=(18, 6), dpi=800)
    fig.tight_layout(pad=3.0)

    # Contour plot for Mean
    c1 = ax[0].contourf(X1, X2, f_mean, levels=100, cmap='viridis')
    fig.colorbar(c1, ax=ax[0])
    ax[0].scatter(X[:, 0], X[:, 1], c='red', marker='x', label='Observed Points')
    ax[0].set_title('Mean Prediction')
    ax[0].set_xlabel('x1')
    ax[0].set_ylabel('x2')

    # Contour plot for LCB
    c2 = ax[1].contourf(X1, X2, LCB, levels=100, cmap='viridis')
    fig.colorbar(c2, ax=ax[1])
    ax[1].scatter(X[:, 0], X[:, 1], c='red', marker='x', label='Observed Points')
    ax[1].set_title('LCB (Lower Confidence Bound)')
    ax[1].set_xlabel('x1')
    ax[1].set_ylabel('x2')

    # Contour plot for EI
    c3 = ax[2].contourf(X1, X2, ei, levels=100, cmap='viridis')
    fig.colorbar(c3, ax=ax[2])
    ax[2].scatter(X[:, 0], X[:, 1], c='red', marker='x', label='Observed Points')
    ax[2].set_title('Expected Improvement')
    ax[2].set_xlabel('x1')
    ax[2].set_ylabel('x2')

     # Save the plot to a file in Google Drive
    plt.savefig(os.path.join(save_dir, f'NoiseLESS_Vanilla_GP_3HC_iteration_{iteration + 1}_2Dplots.png'))
    plt.close()

    print(f"Figure saved for iteration {iteration + 1}")

# After all iterations have completed, save the final results to a CSV file in Google Drive
results_df.to_csv(os.path.join(save_dir, 'NoiseLESS_Final_Vanilla_3HC_2D_GP_Results.csv'), index=False)
print("Final results saved to Final_GP_Results.csv")